LLM - chatGPT, Anthropic 

In [ ]:
import os
from dotenv import load_dotenv

# 환경변수 읽어오기
load_dotenv(override=True)  # .env 파일을 덮어쓰기 모드로 읽기

# 환경변수 불러오기
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

print(f"openai key values ::: {openai_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"anthropic key values ::: {anthropic_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"huggingface_token::: {huggingface_token}")  # 테스트용 (실제 서비스에서는 print 금지)
#...ddd


In [ ]:
from langchain_anthropic import ChatAnthropic
from anthropic import Anthropic
import httpx  

vfy_client = httpx.Client(verify=False)

# 1. 직접 Anthropic
client = Anthropic(api_key=anthropic_key, http_client=vfy_client)

# 2. Langchain Anthropic 모델 호출
chat = ChatAnthropic(
    model_name ="claude-3-opus-20240229",
    anthropic_api_key=anthropic_key,)

chat._client = client

# chat._client._transport._pool._ssl_context.check_hostname = False
# chat._client._transport._pool._ssl_context.verify_mode = 0  # ssl.CERT_NONE
chat.invoke("안녕~ 너를 소개해줄래?")

In [ ]:
import anthropic
import httpx    



print(f"anthropic key values ::: {anthropic_key}")

vfy_client = httpx.Client(verify=False)

client = anthropic.Anthropic(api_key=anthropic_key, http_client=vfy_client)


# client._client._transport._pool._ssl_context.check_hostname = False
# client._client._transport._pool._ssl_context.verify_mode = 0  # ssl.CERT_NONE

# print("client values {} ::: {}".format(type(client), client) ) # 테스트용 (실제 서비스에서는 print 금지)

response = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Anthropic AI 소개해줘"}
    ]
)

print(response.content)

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model_name = 'gpt-4o',
    openai_api_key=openai_key
)
response = chat.invoke("너를 소개해줘")

print(response.content)  # 테스트용 (실제 서비스에서는 print 금지)


# **Document Loader**
## ***<span style="color:yellow">PDF Loader</span>***

In [ ]:
#PyPDF 설치
# !pip install -q pypdf
# G:\내 드라이브\문영호\109 RFP MOH eng.pdf

#PyPDFLoader 불러오기
from langchain_community.document_loaders import PyPDFLoader

# PDF파일 불러올 객체 PyPDFLoader 선언 - window11 
# loader = PyPDFLoader(r"G:\내 드라이브\문영호\109 RFP MOH eng.pdf")
loader = PyPDFLoader(
    r"G:\내 드라이브\LLM-RAG-LangChain\대한민국헌법(헌법)(제00010호)(19880225).pdf"
)

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
# loader = PyPDFLoader(
#     r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
# )

# PDF파일 로드 및 페이지별로 자르기
pages = loader.load_and_split()
print(f"페이지 수: {len(pages)}")
# print(pages[0].page_content)

for i in range(5):   
    print(f"페이지 {i+1} 내용") 
    print("===================================")
    print(pages[i].page_content)
    print(pages[i].metadata)
    print(pages[i].metadata['page'])
    print(pages[i].metadata['page_label'])


# **Text Splitter**

<!-- **굵게 (bold)**  
*기울임 (italic)*  
***굵고 기울임*** -->

# ***<sapn style="color:green">Text Embbeding</span>***
<!-- ## ***글자 크기 조정 2***
###  **글자 크기 조정 3** -->

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

## **openAI Embedding Model**

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#임베딩 모델 API 호출
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

#PDF 문서 로드
# loader = PyPDFium2Loader(r"G:\내 드라이브\문영호\109 RFP MOH eng.pdf")
# loader = PyPDFium2Loader(r"G:\내 드라이브\문영호\알파고\alphago.pdf")
# loader = PyPDFium2Loader(r"G:\내 드라이브\LLM-RAG-LangChain\대한민국헌법(헌법)(제00010호)(19880225).pdf")
loader = PyPDFium2Loader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
# PDF파일 로드 및 페이지별로 자르기
pages = loader.load()

#PDF 문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

texts = text_splitter.split_documents(pages)

#청크 수 확인
print(f"청크 수: {len(texts)}")
#청크 내용 확인
print("started printing chunk contents")
for i in range(1,2):   
    print(f"청크 {i+1} 내용") 
    print("===================================")
    print(texts[i].page_content)
    print(texts[i].metadata)
    print(texts[i].metadata['page'])
    # print(texts[i].metadata['page_label'])
print("finished printing chunk contents")

#OpenAI 임베딩 모델로 청크들을 임베딩 변환하기
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])
len(embeddings), len(embeddings[0])



##### **[문장 유사도]**

In [ ]:
chunk = [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "AI 초보자로서 잘 부탁드립니다.",
        "열심히 배워서 회사의 발전에 이바지할 수 있는 인재로 성장하겠습니다.",
        "LG CNS 만세 저는 아부쟁이입니다."
    ]
embeddings = embeddings_model.embed_documents(chunk)
print(embeddings)

# 임베딩 모델 API 호출 - embed_query
embedded_query_q1 = embeddings_model.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_q2 = embeddings_model.embed_query("이 대화에서 언급된 사람의 포부는 무엇입니까?")
embedded_query_q3 = embeddings_model.embed_query("이 대화에서 언급된 사람은 아부쟁이일까요 아닐까요?")
embedded_query_a = embeddings_model.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

from numpy import dot
from numpy.linalg import norm
import numpy as np

# 임베딩 간 유사도 함수
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

for i in range(len(embeddings)):
    print(chunk[i])
    print(cos_sim(embedded_query_q3, embeddings[i]))


### **오픈소스 임베딩 모델 활용하기**

**[jhgan/ko-sroberta-multitask 임베딩 모델 활용]**

In [ ]:
# pip install sentence-transformers
# pip install --upgrade sentence-transformers  ->  필요

from numpy import dot
from numpy.linalg import norm
import numpy as np

# 임베딩 간 유사도 함수
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

from langchain_community.embeddings import HuggingFaceEmbeddings

#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
# model_name = "jhgan/ko-sroberta-multitask"
# 한국어 NLI 기반 SBERT, RAG용으로 최적화된 모델
model_name = "jhgan/ko-sbert-nli"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

examples = ko_embedding.embed_documents(
     [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
     ]
 )

embedded_query_q = ko_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = ko_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, examples[1]))
print(cos_sim(embedded_query_q, examples[3]))

OSError: jhgan/ko-sbert-nli does not appear to have a file named pytorch_model.bin but there is a file for TensorFlow weights. Use `from_tf=True` to load this model from those weights.

**[BAAI/bge-small-en 임베딩 모델 활용 코드]**

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
bge_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

examples = bge_embedding.embed_documents(
     [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
     ]
 )

embedded_query_q = bge_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = bge_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, examples[1]))
print(cos_sim(embedded_query_q, examples[3]))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


OSError: BAAI/bge-small-en does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

## **문서 벡터 저장소, Vector Stores**
### **Langchain-Chroma 문서 저장 및 유사 문서 검색**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

# PDF파일 불러올 객체 PyPDFLoader 선언 - window11 
# loader = PyPDFLoader(r"G:\내 드라이브\문영호\109 RFP MOH eng.pdf")
# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)

pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, openai_embedding)


In [ ]:
query = "대통령의 임기는?"
#유사 문서 검색
docs = db.similarity_search(query)
for doc in docs:
    print("===================================")
    print(doc.page_content)
    print(doc.metadata)
    print(doc.metadata['page'])
    # print(doc.metadata['page_label'])

In [ ]:
#유사 문서 검색 및 유사도 출력
db.similarity_search_with_score(query)

**<span style='color:green'>[벡터DB를 로컬 디스크에 저장하고 로드하기]</span>**

In [ ]:
Chroma().delete_collection()

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)


#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)

#save to local disk,
db2 = Chroma.from_documents(docs, ko_embedding, persist_directory="./chroma_db")

In [ ]:
# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=ko_embedding)

query = "대통령의 임기는?"
result = db3.similarity_search(query)
print(result[0].page_content)

# # load from disk
# db3 = Chroma(persist_directory="./chroma_db", embedding_function=ko_embedding)

# query = "대통령의 임기는?"
# result = db3.similarity_search(query)
# print(result[0].page_content)

## **RAG의 핵심, 문서 검색기 Retriever**
### **Retriever의 기본형, 벡터DB 기반 Retriever**
**<span style='color:yellow'>Chroma 벡터 DB 기반 기본 유사 문서 검색</span>**

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

#헌법 PDF 파일 로드
# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

#Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")



**(1) 검색 결과 수 및 조정**

In [ ]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 10})
retriever.invoke("국회의원의 의무")

**(2) 검색 방식 변경 - MMR**

MMR은 쿼리에 대한 (1) 각 문서의 유사성 점수와 (2) 이미 선택된 문서들과의 다양성점수를 조합하여, 각 문서의 최종 점수를 계산합니다.

$\text{MMR} = \lambda \cdot \text{Sim}(d, Q) - (1 - \lambda) \cdot \max_{d' \in D'} \text{Sim}(d, d')$

즉 유사 문서 후보군 중 특정 문서가 후보군 내 문서들 간의 유사성은 낮고, 쿼리와의 유사성이 높은 경우 점수가 높습니다.

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
#헌법 PDF 파일 로드
# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))


In [ ]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
result = retriever.invoke("국회의원의 의무")
for idx, value in enumerate(result):
  print(f"{idx+1}번째 유사 문서:")
  print(value.page_content[:100])
  print("\n\n")

**<span style='color:yellow'>일반 유사도 검색 방식**</span>

In [ ]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
result = retriever.invoke("국회의원의 의무")
for idx, value in enumerate(result):
  print(f"{idx+1}번째 유사 문서:")
  print(value.page_content[:100])
  print("\n\n")